# Sentiment Analysis

In this exercise, we're going to perform sentiment analysis on tweets.

In [1]:
import torch

## Data

We're going to use a truncated version of the [Sentiment140 dataset](https://www.kaggle.com/datasets/kazanova/sentiment140). The full dataset contains 1.6M tweets with a sentiment (polarity) label. Each line of the CSV file contains the following 6 fields:
1. target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
2. ids: The id of the tweet (2087)
3. date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
4. flag: The query (lyx). If there is no query, then this value is NO_QUERY.
5. user: the user that tweeted (robotickilldozr)
6. text: the text of the tweet (Lyx is cool)

We load the truncated CSV file `tweets.2000.processed.noemoticon.csv` instead. It contains 2000 tweets (1 tweet per line), 1000 each for label 0 and 4.

In [2]:
import csv
fieldnames = ['label', 'id', 'date', 'query', 'user', 'text']
with open('tweets.2000.processed.noemoticon.csv', 'r', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=fieldnames)
    for row in reader:
        print(f"Label: {row['label']}")
        print(f"User: {row['user']}")
        print(f"Text: {row['text']}")
        break
print(f"Label is of type {type(row['label'])}, convert to int like this: {int(row['label'])}")

Label: 0
User: _TheSpecialOne_
Text: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Label is of type <class 'str'>, convert to int like this: 0


## Model

We're going to use a popular sentiment analysis model from the CardiffNLP group, which we download from HuggingFace: `cardiffnlp/twitter-roberta-base-sentiment`. As you can see from the name, it is based on a pretrained RoBERTa model and was finetuned on tweets for sentiment analysis. Check out the [model description](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=I+like+turtles.) for examples of how to use it.

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model.eval()
pass  # suppress output

From the model description, we see that we have to preprocess tweets first to bring them into the format expected by the model. Let's implement the `preprocess` method as it is described in their example.

In [4]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

Let's classify the first tweet in our dataset. Give the probabilities of the sentiment classification model for each of the 3 classes it outputs: negative, neutral and positive sentiment.

In [5]:
with open('tweets.2000.processed.noemoticon.csv', 'r', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=fieldnames)
    for row in reader:
        tweet = row['text']
        break

text = preprocess(tweet)
encoded_input = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    output = model(**encoded_input)
probabilities = torch.softmax(output.logits[0], dim=-1)
print(f"Tweet: {tweet}")
print(f"Preprocessed text: {text}")
print(f"Probability of negative sentiment: {probabilities[0]:.2%}")
print(f"Probability of neutral sentiment: {probabilities[1]:.2%}")
print(f"Probability of positive sentiment: {probabilities[2]:.2%}")

Tweet: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Preprocessed text: @user http - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Probability of negative sentiment: 63.41%
Probability of neutral sentiment: 31.54%
Probability of positive sentiment: 5.05%


Now classify all the tweets and evaluate how well the model does in terms of classification accuracy.

In [6]:
def classify(tweet):
    """ Returns probabilities for each of the 3 classes: negative, neutral, positive. """
    text = preprocess(tweet)
    encoded_input = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_input)
    probabilities = torch.softmax(output.logits[0], dim=-1)
    return probabilities

In [7]:
def score(label, predictions):
    """ Returns 1 if the label has the highest probability. """
    # dataset's labels are 0 for negative and 4 for positive --> convert to 0 and 2
    true_label = label / 2
    predicted_label = predictions.argmax().item()
    return 1 if true_label == predicted_label else 0

In [8]:
import tqdm

correct = 0
total = 0
with open('tweets.2000.processed.noemoticon.csv', 'r', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile, fieldnames=fieldnames)
    for row in tqdm.tqdm(reader):
        label = int(row['label'])
        tweet = row['text']
        predictions = classify(tweet)
        correct += score(label, predictions)
        total += 1

print(f"Accuracy: {correct/total:.2%} ({correct}/{total})")

2000it [01:36, 20.69it/s]

Accuracy: 58.40% (1168/2000)
